In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate as PT
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(
    temperature=0.1,  
    max_tokens=50,
    model_name="gpt-4o", 
    
)
prompt = PT.from_template("{goods}에 대하여 짧고 강렬한 광고 문구를 1문장으로 만들어줘")
chain = prompt | model | StrOutputParser()


In [5]:
for token in chain.stream({"goods":"티라노사우로스 잠옷"}):
    print(token, end="", flush=True)

"티라노사우로스 잠옷으로 꿈속에서도 용감한 공룡 모험을 시작하세요!"

In [9]:
chain.invoke({"goods": "샤넬에서 만든 고품격 이쑤시개"})

'"샤넬의 우아함을 한 점에 담다 - 고품격 이쑤시개로 일상의 품격을 높이세요."'

In [11]:
chain.batch([{"goods" : "차두리가 쓰는 탈모방지 샴푸"},{"goods" : "잠이 잘 오는 커피"}])

['"차두리도 선택한 탈모방지 샴푸, 당신의 자신감을 되찾아드립니다!"', '"잠이 솔솔, 꿈까지 부드러운 커피 한 잔!"']

In [13]:
chain.batch(
    [
        {"goods": "페라리"},
        {"goods": "람보르기니"},
        {"goods": "BMW"},
        {"goods": "유모차"},
        {"goods": "벤츠"},
    ],
    config={"max_concurrency": 3},
)

['"페라리: 당신의 심장을 뛰게 할 단 하나의 엔진!"',
 '"람보르기니: 한계를 넘어선 속도, 꿈을 현실로!"',
 '"BMW: 당신의 열정을 도로 위에 펼치세요."',
 '"당신의 소중한 순간을 함께하는, 가장 안전하고 편안한 유모차!"',
 '"벤츠: 당신의 여정을 품격으로 채우다."']

In [14]:
async for token in chain.astream({"goods": "말린 망고"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

"자연의 달콤함을 한 입에, 말린 망고로 상큼한 에너지를 충전하세요!"

In [16]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리합니다.
my_process = chain.ainvoke({"goods": "당도 없는 수박"})
await my_process

C:\Users\GoodusData\AppData\Local\Temp\ipykernel_16004\242177972.py:2: RuntimeWarning: coroutine 'RunnableSequence.ainvoke' was never awaited
  my_process = chain.ainvoke({"goods": "당도 없는 수박"})


'"자연 그대로의 청량함, 당도 없는 수박의 신선한 매력을 경험하세요!"'

In [18]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
my_abatch_process = chain.abatch(
    [
        {"goods": "시노부 피규어"}, 
        {"goods": "몽돌 키보드"}, 
        {"goods": "베트남 여행패키지지"}, 
    ]
)

await my_abatch_process

C:\Users\GoodusData\AppData\Local\Temp\ipykernel_16004\4000541879.py:2: RuntimeWarning: coroutine 'RunnableSequence.abatch' was never awaited
  my_abatch_process = chain.abatch(


['"시노부 피규어로 당신의 공간에 매혹적인 애니메이션의 세계를 불어넣으세요!"',
 '"몽돌 키보드로 타이핑의 예술을 경험하세요!"',
 '"베트남의 숨겨진 보석을 발견하는 여정, 지금 바로 떠나세요!"']

In [19]:
from langchain_core.runnables import RunnableParallel

# {country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = (
    PT.from_template("{country} 의 수도는 어디야?")
    | model
    | StrOutputParser()
)

# {country} 의 면적을 물어보는 체인을 생성합니다.
chain2 = (
    PT.from_template("{country} 의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성합니다.
combined = RunnableParallel(capital=chain1, area=chain2)



chain1.invoke({"country":"대한민국"})

'대한민국의 수도는 서울입니다.'

In [20]:
chain2.invoke({"country":"대한민국"})

'대한민국의 총 면적은 약 100,210 평방 킬로미터입니다. 이 면적에는 한반도의 남쪽 부분과 여러 부속 도서가 포함됩니다.'

In [22]:
combined.invoke({"country" : "남수단"})

{'capital': '남수단의 수도는 주바입니다.',
 'area': '남수단의 면적은 약 619,745 제곱킬로미터입니다. 이는 아프리카 대륙에서 비교적 큰 나라 중 하나로, 2011년에 수단으로부터 독립하여 세계'}

In [23]:
combined.batch([{"country": "대한민국"}, {"country": "미국"}])

[{'capital': '대한민국의 수도는 서울입니다.',
  'area': '대한민국의 총 면적은 약 100,210 평방 킬로미터입니다. 이 면적에는 한반도의 남쪽 부분과 여러 부속 도서가 포함됩니다.'},
 {'capital': '미국의 수도는 워싱턴 D.C.입니다.',
  'area': '미국의 총 면적은 약 983만 평방킬로미터(약 379만 평방마일)입니다. 이 면적에는 50개 주와 워싱턴 D.C., 그리고 미국의 여러 해외 영'}]